In [27]:
import pandas as pd

train_path = '../dataset/PIZZA_train.json'
test_path = '../dataset/PIZZA_dev.json'

In [28]:
import pandas as pd
df = pd.read_json(train_path, lines=True)
dev = pd.read_json(test_path, lines=True)

In [29]:
df.describe()

,train.SRC,train.EXR,train.TOP,train.TOP-DECOUPLED
count,2456446,2456446,2456446,2456446
unique,2456446,694346,2456446,1425035
top,can i have a large bbq pulled pork,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZ...,(ORDER (PIZZAORDER (NUMBER three ) (NOT (TOPPI...
freq,1,1999,1,167


In [30]:
# Identify unique patterns in 'train.EXR'
unique_patterns = df['train.EXR'].value_counts()
unique_patterns.describe()

count    694346.000000
mean          3.537784
std          17.072263
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max        1999.000000
Name: count, dtype: float64

In [31]:
# Select a sample from each pattern
reduced_dataset = df[df['train.EXR'].isin(unique_patterns.index[:5000])] # reduce to 5000 patterns
reduced_dataset.describe()

,train.SRC,train.EXR,train.TOP,train.TOP-DECOUPLED
count,720617,720617,720617,720617
unique,720617,5000,720617,136030
top,can i have a large bbq pulled pork,(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZ...,(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZ...,(ORDER (PIZZAORDER (NUMBER three ) (NOT (TOPPI...
freq,1,1999,1,167


In [32]:
from transformers import AutoTokenizer

# Example: Using a pre-trained tokenizer like BERT
src_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tgt_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

src_text = "can i have a large bbq pulled pork"
#tgt_text = "(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING bbq pulled pork ) ) )"
tgt_text = "(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE LARGE ) (TOPPING BBQ_PULLED_PORK ) ) )"
# Tokenize the input and output
src_tokens = src_tokenizer(src_text, return_tensors="pt", padding=True, truncation=True)
tgt_tokens = tgt_tokenizer(tgt_text, return_tensors="pt", padding=True, truncation=True)

print(src_tokens)
print(tgt_tokens)

C:\Users\Hima\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'input_ids': tensor([[  101,  2064,  1045,  2031,  1037,  2312, 22861,  4160,  2766, 15960,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[  101,  1006,  2344,  1006, 10733,  8551,  2121,  1006,  2193,  1015,
          1007,  1006,  2946,  2312,  1007,  1006, 22286, 22861,  4160,  1035,
          2766,  1035, 15960,  1007,  1007,  1007,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])}


In [33]:
import torch
from torch.utils.data import Dataset, DataLoader

class PizzaDataset(Dataset):
    def __init__(self, data, src_tokenizer, tgt_tokenizer):
        self.data = data
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_text = self.data["train.SRC"][idx]
        tgt_text = self.data["train.EXR"][idx]

        src_tokens = self.src_tokenizer(src_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)
        tgt_tokens = self.tgt_tokenizer(tgt_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)

        return {
            "src_input_ids": src_tokens["input_ids"].squeeze(0),
            "src_attention_mask": src_tokens["attention_mask"].squeeze(0),
            "tgt_input_ids": tgt_tokens["input_ids"].squeeze(0),
            "tgt_attention_mask": tgt_tokens["attention_mask"].squeeze(0),
        }


train_dataset = PizzaDataset(reduced_dataset.reset_index(), src_tokenizer, tgt_tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [34]:
train_dataset.__getitem__(1)

{'src_input_ids': tensor([  101,  2283,  2946, 11812, 19116, 11345,  2007,  2137,  8808,  1998,
          2007, 18565,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'src_attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 'tgt_input_ids': tensor([  101,  1006,  2344,  1006, 10733,  8551,  2121,  1006,  2193,  1015,
          1007,  1006,  2946,  2283,  1035,  2946,  1007,  1006,  2806, 11812,
          1035, 19116,  1007,  1006, 22286,  2137,  1035,  8808,  1007,  1006,
         22286, 23827,  1007,  1007,  1007,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [35]:
class DevPizzaDataset(Dataset):
    def __init__(self, data, src_tokenizer, tgt_tokenizer):
        self.data = data
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_text = self.data["dev.SRC"][idx]
        tgt_text = self.data["dev.EXR"][idx]

        src_tokens = self.src_tokenizer(src_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)
        tgt_tokens = self.tgt_tokenizer(tgt_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)

        return {
            "src_input_ids": src_tokens["input_ids"].squeeze(0),
            "src_attention_mask": src_tokens["attention_mask"].squeeze(0),
            "tgt_input_ids": tgt_tokens["input_ids"].squeeze(0),
            "tgt_attention_mask": tgt_tokens["attention_mask"].squeeze(0),
        }
val_dataset = DevPizzaDataset(dev, src_tokenizer, tgt_tokenizer)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [36]:
val_dataset.__getitem__(3)

{'src_input_ids': tensor([  101,  1045,  1005,  1040,  2066,  2000,  2344,  1037,  2312, 20949,
          1998, 11565, 10733,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'src_attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 'tgt_input_ids': tensor([  101,  1006,  2344,  1006, 10733,  8551,  2121,  1006,  2193,  1015,
          1007,  1006,  2946,  2312,  1007,  1006, 22286, 24444,  1007,  1006,
         22286, 23582,  1007,  1007,  1007,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [37]:
len(val_dataset.__getitem__(3)['src_input_ids'])

50

### BiLSTM-Based Transformer

In [38]:
import torch
import torch.nn as nn
from tqdm import tqdm

class BiLSTMEncoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, vocab_size, num_layers=1):
        super(BiLSTMEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = True
        self.num_directions = 2 if self.bidirectional else 1
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers,
                            bidirectional=self.bidirectional, batch_first=True)

    def forward(self, src, src_lengths):
        embedded = self.embedding(src)  # (batch_size, seq_len, embedding_dim)

        # Initialize hidden and cell states
        batch_size = src.size(0)
        h0 = torch.zeros(self.num_layers * self.num_directions, batch_size, self.hidden_dim).to(src.device)
        c0 = torch.zeros(self.num_layers * self.num_directions, batch_size, self.hidden_dim).to(src.device)

        # Pack the sequences for the LSTM
        src_lengths = src_lengths.cpu()  # Move lengths to CPU
        packed = nn.utils.rnn.pack_padded_sequence(embedded, src_lengths, batch_first=True, enforce_sorted=False)

        # Pass through the LSTM
        outputs, (hidden, cell) = self.lstm(packed, (h0, c0))
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)

        # Return all hidden states (both forward and backward)
        return outputs, hidden, cell



class LSTMDecoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, vocab_size, num_layers=1):
        super(LSTMDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(1)  # (batch_size, 1)
        embedded = self.embedding(input)  # (batch_size, 1, embedding_dim)
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(1))  # (batch_size, vocab_size)
        return prediction, hidden, cell


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, src_lengths, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        tgt_len = tgt.shape[1]
        tgt_vocab_size = self.decoder.fc_out.out_features

        # Tensor to store decoder outputs
        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        # Encode source sequence
        encoder_outputs, hidden, cell = self.encoder(src, src_lengths)

        # Reshape hidden and cell states for the decoder
        hidden = hidden.view(self.encoder.num_layers, batch_size, -1)
        cell = cell.view(self.encoder.num_layers, batch_size, -1)

        # First input to the decoder is the <sos> token
        input = tgt[:, 0]

        for t in range(1, tgt_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t, :] = output
            # Decide whether to use teacher forcing
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)  # Get the highest predicted token
            input = tgt[:, t] if teacher_force else top1

        return outputs

def save_checkpoint(model, optimizer, epoch, path="checkpoint.pth"):
    checkpoint = {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "epoch": epoch,
    }
    torch.save(checkpoint, path)
    print(f"Checkpoint saved at epoch {epoch}.")

def load_checkpoint(model, optimizer, path="checkpoint.pth"):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    epoch = checkpoint["epoch"]
    print(f"Checkpoint loaded. Resuming from epoch {epoch}.")
    return epoch


def train_model(model, dataloader, optimizer, criterion, device, checkpoint_path):
    model.train()
    epoch_loss = 0
    total_batches = len(dataloader)
    progress_bar = tqdm(dataloader, desc="Training Progress", unit="batch", leave=True)
    for batch_idx, batch in enumerate(progress_bar):
        src = batch["src_input_ids"].to(device)
        src_lengths = (batch["src_attention_mask"] != 0).sum(dim=1).cpu()
        tgt = batch["tgt_input_ids"].to(device)

        optimizer.zero_grad()
        output = model(src, src_lengths, tgt)
        # Reshape for loss computation
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        tgt = tgt[:, 1:].reshape(-1)

        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        avg_loss = epoch_loss / (batch_idx + 1)
        progress_bar.set_description(f"Training Progress: Batch {batch_idx + 1}/{total_batches}, Avg Loss: {avg_loss:.4f}")


    return epoch_loss / len(dataloader)

def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            src_lengths = (batch["src_attention_mask"] != 0).sum(dim=1).to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src, src_lengths, tgt, teacher_forcing_ratio=0)  # No teacher forcing during evaluation
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            tgt = tgt[:, 1:].reshape(-1)

            loss = criterion(output, tgt)
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)


### Token-Level Accuracy

In [ ]:
def evaluate_model_with_accuracy(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    total_tokens = 0
    correct_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            src_lengths = (batch["src_attention_mask"] != 0).sum(dim=1).to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src, src_lengths, tgt, teacher_forcing_ratio=0)  # No teacher forcing during evaluation
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)  # Reshape for token comparison
            tgt = tgt[:, 1:].reshape(-1)

            # Calculate loss
            loss = criterion(output, tgt)
            epoch_loss += loss.item()

            # Calculate accuracy
            predictions = output.argmax(dim=1)  # Get the index of the max log-probability
            correct_tokens += (predictions == tgt).sum().item()
            total_tokens += tgt.size(0)

    accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0
    return epoch_loss / len(dataloader), accuracy


### Sequence-Level Accuracy

In [ ]:
def evaluate_model_with_sequence_accuracy(model, dataloader, criterion, device, tokenizer):
    model.eval()
    epoch_loss = 0
    total_sequences = 0
    correct_sequences = 0

    with torch.no_grad():
        for batch in dataloader:
            src = batch["src_input_ids"].to(device)
            src_lengths = (batch["src_attention_mask"] != 0).sum(dim=1).to(device)
            tgt = batch["tgt_input_ids"].to(device)

            output = model(src, src_lengths, tgt, teacher_forcing_ratio=0)  # No teacher forcing during evaluation
            output_dim = output.shape[-1]
            output = output.argmax(dim=-1)  # Get the predicted tokens

            # Decode sequences for comparison
            predicted_sequences = [tokenizer.decode(seq, skip_special_tokens=True) for seq in output]
            target_sequences = [tokenizer.decode(seq, skip_special_tokens=True) for seq in tgt]

            # Calculate sequence accuracy
            for pred, tgt in zip(predicted_sequences, target_sequences):
                if pred == tgt:
                    correct_sequences += 1
                total_sequences += 1

    sequence_accuracy = correct_sequences / total_sequences if total_sequences > 0 else 0
    return epoch_loss / len(dataloader), sequence_accuracy


### Initialize and Train

In [39]:
INPUT_DIM = len(src_tokenizer.vocab)
OUTPUT_DIM = len(tgt_tokenizer.vocab)
EMBEDDING_DIM = 256
HIDDEN_DIM = 512
NUM_LAYERS = 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = BiLSTMEncoder(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, INPUT_DIM, NUM_LAYERS).to(DEVICE)
decoder = LSTMDecoder(EMBEDDING_DIM, HIDDEN_DIM * 2, OUTPUT_DIM, OUTPUT_DIM, NUM_LAYERS).to(DEVICE)
model = Seq2Seq(encoder, decoder, DEVICE).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=tgt_tokenizer.pad_token_id)

checkpoint_path = "checkpoint_1.pth"
start_epoch = 1
num_epochs = 1

for epoch in range(start_epoch, num_epochs + 1):
    epoch_loss = 0
    total_batches = len(train_dataloader)
    train_loss = train_model(model, train_dataloader, optimizer, criterion, DEVICE, checkpoint_path)
    val_loss = evaluate_model(model, val_dataloader, criterion, DEVICE)
    print(f"Train Loss: {train_loss:.3f}, Val Loss: {val_loss:.3f}")
    save_checkpoint(model, optimizer, epoch, checkpoint_path)


Training Progress: Batch 11260/11260, Avg Loss: 0.5588: 100%|██████████| 11260/11260 [2:50:46<00:00,  1.10batch/s]


Train Loss: 0.559, Val Loss: 5.866
Checkpoint saved at epoch 1.


In [42]:
evaluate_model_with_accuracy(model, val_dataloader, criterion, DEVICE)

(5.866480429967244, 0.228477597935726)

In [27]:
import os

if os.path.exists(checkpoint_path):
    load_checkpoint(model, optimizer, checkpoint_path)

# Train the model from the loaded epoch
train_model(
    model,
    train_dataloader,
    optimizer,
    criterion,
    DEVICE,
    checkpoint_path=checkpoint_path
)

Checkpoint loaded. Resuming from epoch {'src_input_ids': tensor([[  101,  2064,  1045,  ...,     0,     0,     0],
        [  101,  1045,  1005,  ...,     0,     0,     0],
        [  101,  1045,  2215,  ...,     0,     0,     0],
        ...,
        [  101,  1045,  1005,  ...,     0,     0,     0],
        [  101,  2176, 10733,  ...,     0,     0,     0],
        [  101,  2176, 10733,  ...,     0,     0,     0]]), 'src_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'tgt_input_ids': tensor([[  101,  1006,  2344,  ...,     0,     0,     0],
        [  101,  1006,  2344,  ...,     0,     0,     0],
        [  101,  1006,  2344,  ...,     0,     0,     0],
        ...,
        [  101,  1006,  2344,  ...,   102,     0,     0],
        [  101,  1006,  2344,  ..., 13874, 14580,   102],
        [  101,  10

Training Progress:   0%|          | 0/38382 [00:24<?, ?batch/s]


KeyboardInterrupt: 

### Inference

In [ ]:
def generate(model, src, src_lengths, max_len=50):
    model.eval()
    with torch.no_grad():
        encoder_outputs, hidden, cell = model.encoder(src, src_lengths)
        input = torch.tensor([tgt_tokenizer.bos_token_id]).to(model.device)

        outputs = []
        for _ in range(max_len):
            output, hidden, cell = model.decoder(input, hidden, cell)
            top1 = output.argmax(1).item()
            outputs.append(top1)
            if top1 == tgt_tokenizer.eos_token_id:
                break
            input = torch.tensor([top1]).to(model.device)

        return tgt_tokenizer.decode(outputs, skip_special_tokens=True)


In [ ]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    print(f"Model loaded from {path}")
    return model

In [ ]:
save_model(model, "../weights/transformer_Bilstm.pt")